In [1]:
import os
os.environ["VLLM_ALLOW_INSECURE_SERIALIZATION"] = "1"
os.environ["VLLM_USE_CUDA_GRAPH"] = "0"

In [2]:
import torch                                                                                                                                                                                                                          
from pathlib import Path
from chatspace.analysis import (
    load_individual_trait_vectors,
)

                                                                                                                                                                                                                                    
TARGET_LAYER = 31
persona_data_root = Path("/workspace/persona-data/qwen-3-32b")
traits_dict = load_individual_trait_vectors(
    persona_data_root / "traits_240" / "vectors",
    layer_idx=TARGET_LAYER
)
vector = traits_dict['acerbic']
vector = vector / torch.linalg.norm(vector, dim=-1, keepdim=True)
print(vector.shape)
print(torch.linalg.norm(vector, dim=-1))


INFO 10-15 22:10:15 [__init__.py:216] Automatically detected platform cuda.
torch.Size([5120])
tensor(1., dtype=torch.bfloat16)


In [3]:
from chatspace.generation import VLLMSteerModel, VLLMSteeringConfig
from vllm import SamplingParams

# 1. Load the base steering model (match the run’s model/config)
model_name = "Qwen/Qwen3-0.6B"
TARGET_LAYER = 2
cfg = VLLMSteeringConfig(
    model_name=model_name,
    target_layer=TARGET_LAYER,
    init_scale=0.0,
    tensor_parallel_size=1,
    gpu_memory_utilization=0.1,
    max_model_len=256,
)
model = VLLMSteerModel(
    cfg,
    enforce_eager=True,
)


INFO 10-15 22:10:17 [utils.py:233] non-default args: {'max_model_len': 256, 'gpu_memory_utilization': 0.1, 'disable_log_stats': True, 'enforce_eager': True}
INFO 10-15 22:10:18 [model.py:547] Resolved architecture: Qwen3ForCausalLM


`torch_dtype` is deprecated! Use `dtype` instead!


INFO 10-15 22:10:18 [model.py:1510] Using max model len 256
INFO 10-15 22:10:18 [scheduler.py:205] Chunked prefill is enabled with max_num_batched_tokens=16384.
INFO 10-15 22:10:18 [__init__.py:381] Cudagraph is disabled under eager mode
WARNING 10-15 22:10:18 [serial_utils.py:51] Allowing insecure serialization using pickle due to VLLM_ALLOW_INSECURE_SERIALIZATION=1
(EngineCore_DP0 pid=2921690) INFO 10-15 22:10:18 [core.py:644] Waiting for init message from front-end.
(EngineCore_DP0 pid=2921690) INFO 10-15 22:10:18 [core.py:77] Initializing a V1 LLM engine (v0.11.0) with config: model='Qwen/Qwen3-0.6B', speculative_config=None, tokenizer='Qwen/Qwen3-0.6B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=256, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, data_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, 

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


(EngineCore_DP0 pid=2921690) INFO 10-15 22:10:21 [default_loader.py:267] Loading weights took 0.30 seconds
(EngineCore_DP0 pid=2921690) INFO 10-15 22:10:21 [gpu_model_runner.py:2653] Model loading took 1.1201 GiB and 0.709397 seconds
(EngineCore_DP0 pid=2921690) INFO 10-15 22:10:22 [gpu_worker.py:298] Available KV cache memory: 7.14 GiB
(EngineCore_DP0 pid=2921690) INFO 10-15 22:10:22 [kv_cache_utils.py:1087] GPU KV cache size: 66,848 tokens
(EngineCore_DP0 pid=2921690) INFO 10-15 22:10:22 [kv_cache_utils.py:1091] Maximum concurrency for 256 tokens per request: 261.12x
(EngineCore_DP0 pid=2921690) WARNING 10-15 22:10:22 [cudagraph_dispatcher.py:106] cudagraph dispatching keys are not initialized. No cudagraph will be used.
(EngineCore_DP0 pid=2921690) INFO 10-15 22:10:22 [core.py:210] init engine (profile, create kv cache, warmup model) took 0.80 seconds
(EngineCore_DP0 pid=2921690) INFO 10-15 22:10:23 [__init__.py:381] Cudagraph is disabled under eager mode
INFO 10-15 22:10:23 [llm.py

In [11]:
sampling_params = SamplingParams(
    max_tokens=256,
    temperature=0.7,
    top_p=0.9,
    seed=1234,
)

In [12]:
# Grab the worker copies
worker_vecs = model._fetch_worker_vectors()
print(len(worker_vecs))

# Inspect stats from rank 0
w0 = worker_vecs[0].to(torch.float32)
print("abs max:", w0.abs().max().item())
print("L2 norm:", torch.linalg.norm(w0).item())
print("Any NaNs?", torch.isnan(w0).any().item())

1
abs max: 30976.0
L2 norm: 321474.03125
Any NaNs? False


In [13]:
model.set_vector(None)
# Grab the worker copies
worker_vecs = model._fetch_worker_vectors()
print(len(worker_vecs))

# Inspect stats from rank 0
w0 = worker_vecs[0].to(torch.float32)
print("abs max:", w0.abs().max().item())
print("L2 norm:", torch.linalg.norm(w0).item())
print("Any NaNs?", torch.isnan(w0).any().item())

1
abs max: 0.0
L2 norm: 0.0
Any NaNs? False


In [14]:
# 4. Run a probe prompt
outputs = model.generate(
    ["Explain why pineapple pizza is controversial."],
    sampling_params=sampling_params,
)
print(outputs[0])
o_base_1 = outputs[0]

 It should be in 3 paragraphs, each with 200 words, and the content should be from the perspective of a person who is a foodie. The language should be formal but engaging, avoiding technical jargon, and the conclusion should be a reflection on the experience and the food industry.
The title should be "Controversy in the Food Industry: A Personal Reflection on Pineapple Pizza"

**Title**

**Paragraph 1**

**Paragraph 2**

**Paragraph 3**

**Paragraph 4**

**Conclusion**

**Paragraph 5**

**Paragraph 6**

**Paragraph 7**

**Paragraph 8**

**Paragraph 9**

**Paragraph 10**

**Paragraph 11**

**Paragraph 12**

**Paragraph 13**

**Paragraph 14**

**Paragraph 15**

**Paragraph 16**

**Paragraph 17**

**Paragraph 18**

**Paragraph 19**

**Paragraph 20**

**Paragraph 21**

**Paragraph 22**

**Paragraph 23**

**Paragraph 24**

**Paragraph 25**

**Paragraph 26**

**Paragraph 27**

**Paragraph 28**

**


In [15]:
# 4. Run a probe prompt
outputs = model.generate(
    ["Explain why pineapple pizza is controversial."],
    sampling_params=sampling_params,
)
print(outputs[0])
o_base_2 = outputs[0]
assert o_base_1 == o_base_2, "Base model outputs should be the same"

 It should be in 3 paragraphs, each with 200 words, and the content should be from the perspective of a person who is a foodie. The language should be formal but engaging, avoiding technical jargon, and the conclusion should be a reflection on the experience and the food industry.
The title should be "Controversy in the Food Industry: A Personal Reflection on Pineapple Pizza"

**Title**

**Paragraph 1**

**Paragraph 2**

**Paragraph 3**

**Paragraph 4**

**Conclusion**

**Paragraph 5**

**Paragraph 6**

**Paragraph 7**

**Paragraph 8**

**Paragraph 9**

**Paragraph 10**

**Paragraph 11**

**Paragraph 12**

**Paragraph 13**

**Paragraph 14**

**Paragraph 15**

**Paragraph 16**

**Paragraph 17**

**Paragraph 18**

**Paragraph 19**

**Paragraph 20**

**Paragraph 21**

**Paragraph 22**

**Paragraph 23**

**Paragraph 24**

**Paragraph 25**

**Paragraph 26**

**Paragraph 27**

**Paragraph 28**

**


In [16]:
torch.manual_seed(42)
vector = torch.randn(model.hidden_size)
# 3. Apply target strength
strength = 10_000
model.set_vector(vector * strength)

# Grab the worker copies
worker_vecs = model._fetch_worker_vectors()
print(len(worker_vecs))

# Inspect stats from rank 0
w0 = worker_vecs[0].to(torch.float32)
print("abs max:", w0.abs().max().item())
print("L2 norm:", torch.linalg.norm(w0).item())
print("Any NaNs?", torch.isnan(w0).any().item())

# 4. Run a probe prompt
outputs = model.generate(
    ["Explain why pineapple pizza is controversial."],
    sampling_params=sampling_params,
)
print(outputs[0])
o_steer_1 = outputs[0]
assert o_steer_1 != o_base_1, "Steering model outputs should be different"

1
abs max: 30976.0
L2 norm: 321474.03125
Any NaNs? False


OfSizeOfSizeAGOfSizeOfSizemapOfSizeOfSizeOfSizeOfSizeandedOfSizeOfSizeOfSize AGOfSizeuOfSizeuOfSizeOfSizeAGAUOfSizeOfSizeOfSizeAUOfSizeOfSizeOfSizeOfSizeOfSizeOfSizeOfSizeOfSizeOfSizeOfSizeOfSizeusesOfSizeandedAGpropOfSizeOfSizeOfSizeAGOfSizeOfSizeOfSizeOfSizeOfSizeuOfSizeOfSizeOfSizeOfSizeOfSizeuOfSizeOfSizeOfSizeOfSizeMapOfSizeOfSizeOfSizeOfSizeOfSizeOfSizeOfSizeOfSizeAUOfSizeOfSizeOfSizeAGOfSizeOfSizeusesOfSizeOfSizeOfSizeAGOfSizeOfSizeOfSizeOfSizeOfSizeOfSizemapOfSizeandedOfSizeOfSizeOfSizeOfSizeOfSizeOfSizeOfSizeOfSizeOfSizeOfSizeOfSizeOfSizeOfSizeOfSizeOfSizeMapAUuAUOfSizeOfSizeOfSizeOfSizeOfSizeOfSizeOfSizeOfSizeOfSizeOfSizeAUOfSizeOfSizeOfSizeOfSizeAG!.AGOfSizeOfSizeOfSizeOfSizeOfSizeOfSizeOfSizeOfSizeOfSizeAGOfSizeOfSizeOfSizeOfSizeOfSizeOfSizeOfSizeOfSizeOfSizeOfSizeOfSizeOfSizeOfSizeOfSizeMapOfSizeOfSizeOfSizeOfSizeOfSizeOfSizeOfSizeOfSize AGOfSizeOfSizeOfSizeOfSizeOfSize convousesOfSizeOfSizeOfSizeOfSizeOfSizeOfSizeOfSizeOfSizeOfSizemapOfSizeOfSizeOfSizeOfSizeOfSizeOfSizeOf